In [3]:
import numpy as np
import pandas as pd

In [35]:
df_soap_reduced20 = pd.read_pickle("df_soap_host_217.pkl")

In [5]:
df = pd.read_pickle("defect_df.p")
df = df[df["defect_formation_energy"] > 0]
y = df["defect_formation_energy"]

In [6]:
def corr_reduction(corr, thresh, data):    
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= thresh:
                if columns[j]:
                    columns[j] = False
    selected_columns = data.columns[columns]
    data = data[selected_columns]
    return data

# SOAP + CM

In [5]:
df_cm = pd.read_pickle("defect_df_cm_5")
df_cm_reduced = df_cm[df_cm["defect_formation_energy"] > 0]

In [6]:
df_cm_reduced = df_cm_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

df_cm_reduced = corr_reduction(df_cm_reduced.corr(),0.9, df_cm_reduced)

In [36]:
df_soap_cm = df_soap_reduced20.join(df_cm_reduced)

NameError: name 'df_cm_reduced' is not defined

In [34]:
df_soap_cm

NameError: name 'df_soap_cm' is not defined

In [7]:
from sklearn.ensemble import RandomForestRegressor
rf_soap_cv = RandomForestRegressor()
from xgboost import XGBRegressor
xgbr = XGBRegressor()

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [11]:
out_soap_cm = cross_validate(rf_soap_cv, df_soap_cm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

In [12]:
out_cm_gbr = cross_validate(xgbr, df_soap_cm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [13]:
out_soap_cm['test_score'].mean()

-0.6869301648851133

In [14]:
out_cm_gbr['test_score'].mean()

-0.6347897917431555

# SOAP + SCM

In [40]:
df_scm = pd.read_pickle("defect_df_scm_5")
df_scm_reduced = df_scm[df_scm["defect_formation_energy"] > 0]

In [41]:
df_scm_reduced = df_scm_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

df_scm_reduced = corr_reduction(df_scm_reduced.corr(),0.9, df_scm_reduced)

In [42]:
df_soap_scm = df_soap_reduced20.join(df_scm_reduced)

In [18]:
df_soap_scm_reduced = corr_reduction(df_soap_scm.corr(),0.9,df_soap_scm)

In [43]:
df_soap_scm.head()

,mean SOAP_2688,std_dev SOAP_2499,std_dev SOAP_4434,mean SOAP_4924,mean SOAP_4735,mean SOAP_1364,mean SOAP_320,std_dev SOAP_5898,std_dev SOAP_1764,mean SOAP_5470,...,sine coulomb matrix eig 144,sine coulomb matrix eig 181,sine coulomb matrix eig 192,sine coulomb matrix eig 193,sine coulomb matrix eig 216,sine coulomb matrix eig 217,sine coulomb matrix eig 223,sine coulomb matrix eig 293,sine coulomb matrix eig 335,sine coulomb matrix eig 401
1917,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,1644.805984,90.126673,80.986545,80.639932,41.292350,41.336154,42.792877,44.301388,50.295898,53.749478
1827,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.089515,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
352,0.0,0.000000,0.0,0.0,0.000000,0.010073,0.000000,0.0,0.0,-0.016928,...,30.567663,34.606064,75.603930,75.170205,69.783327,75.826760,42.394642,61.545370,53.717163,0.000000
91,0.0,0.000000,0.0,0.0,-0.088388,0.000000,0.029132,0.0,0.0,0.000000,...,-13.431210,-13.115136,48.319659,48.412045,64.564599,64.267563,71.139325,59.899083,0.000000,0.000000
1868,0.0,0.001776,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,137.943710,72.385616,71.138847,71.166065,51.514002,66.325768,64.909046,56.641210,0.000000,0.000000


In [20]:
# rf_soap_scm = RandomForestRegressor(**{'bootstrap': True,
#  'max_depth': None,
#  'max_features': 3,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 500})
out_soap_scm = cross_validate(rf_soap_cv, df_soap_scm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [21]:
out_scm_gbr = cross_validate(xgbr, df_soap_scm, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [22]:
out_soap_scm['test_score'].mean()

-0.6714947561462322

In [23]:
out_scm_gbr['test_score'].mean()

-0.6384729643015497

# SOAP + SCM + Density

In [24]:
df_d = pd.read_pickle("df_d_mpe_gs.pkl")
df_d_reduced = df_d[df_d["defect_formation_energy"] > 0]

In [25]:
df_d_reduced = df_d_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

# df_cm_reduced = corr_reduction(df_cm_reduced.corr(),0.9, df_cm_reduced)

In [44]:
df_soap_scd = df_soap_scm.join(df_d_reduced)

In [27]:
df_d_reduced

,density,vpa,packing fraction,max packing efficiency,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric
1917,7.340130,11.959952,0.410482,0.300958,1,triclinic,7,False
1827,5.662070,16.106118,0.733155,0.311252,1,triclinic,7,False
352,5.551771,11.513054,0.685517,0.379453,1,triclinic,7,False
91,5.085798,12.389093,0.794698,0.277284,8,monoclinic,6,False
1868,6.587255,15.571968,0.641528,0.265092,1,triclinic,7,False
...,...,...,...,...,...,...,...,...
1638,4.768546,15.467348,0.614265,0.295605,1,triclinic,7,False
1095,3.568996,10.449835,0.733639,0.404477,3,monoclinic,6,False
1130,5.698582,9.643752,0.713843,0.517118,156,trigonal,3,False
1294,5.339617,14.000677,0.885532,0.352660,25,orthorhombic,5,False


In [45]:
df_soap_scd = df_soap_scd.drop(["crystal_system"], axis = 1)

In [49]:


rf_soap_scd = RandomForestRegressor()
out_soap_scd = cross_validate(rf_soap_scd, df_soap_scd, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [50]:
out_soap_scd['test_score'].mean()

-0.6699392681278349

In [51]:
out_scd_gbr = cross_validate(xgbr, df_soap_scd, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [52]:
out_soap_scd['test_score'].mean()

-0.6699392681278349

In [53]:
out_scd_gbr['test_score'].mean()

-0.6353530957015295

In [46]:
df_soap_scd

,mean SOAP_2688,std_dev SOAP_2499,std_dev SOAP_4434,mean SOAP_4924,mean SOAP_4735,mean SOAP_1364,mean SOAP_320,std_dev SOAP_5898,std_dev SOAP_1764,mean SOAP_5470,...,sine coulomb matrix eig 293,sine coulomb matrix eig 335,sine coulomb matrix eig 401,density,vpa,packing fraction,max packing efficiency,spacegroup_num,crystal_system_int,is_centrosymmetric
1917,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,44.301388,50.295898,53.749478,7.340130,11.959952,0.410482,0.300958,1,7,False
1827,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.089515,...,0.000000,0.000000,0.000000,5.662070,16.106118,0.733155,0.311252,1,7,False
352,0.0,0.000000,0.000000,0.000000,0.000000,0.010073,0.000000,0.000000,0.0,-0.016928,...,61.545370,53.717163,0.000000,5.551771,11.513054,0.685517,0.379453,1,7,False
91,0.0,0.000000,0.000000,0.000000,-0.088388,0.000000,0.029132,0.000000,0.0,0.000000,...,59.899083,0.000000,0.000000,5.085798,12.389093,0.794698,0.277284,8,6,False
1868,0.0,0.001776,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,56.641210,0.000000,0.000000,6.587255,15.571968,0.641528,0.265092,1,7,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.0,0.000000,0.006905,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,65.391600,55.273140,61.643390,4.768546,15.467348,0.614265,0.295605,1,7,False
1095,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,3.568996,10.449835,0.733639,0.404477,3,6,False
1130,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,5.698582,9.643752,0.713843,0.517118,156,3,False
1294,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.485143,0.0,0.000000,...,0.000000,0.000000,0.000000,5.339617,14.000677,0.885532,0.352660,25,5,False


# SOAP + CrystalNN

In [9]:
df_c = pd.read_pickle("defect_df_cnn_5")
df_c_reduced = df_c[df_c["defect_formation_energy"] > 0]

In [10]:
df_c_reduced = df_c_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

df_c_reduced = corr_reduction(df_c_reduced.corr(),0.9, df_c_reduced)

In [11]:
df_soap_c = df_soap_reduced20.join(df_c_reduced)

In [12]:
df_soap_c.head()

,mean SOAP_2688,std_dev SOAP_2499,std_dev SOAP_4434,mean SOAP_4924,mean SOAP_4735,mean SOAP_1364,mean SOAP_320,std_dev SOAP_5898,std_dev SOAP_1764,mean SOAP_5470,...,mean wt CN_16,mean wt CN_17,mean wt CN_18,mean wt CN_19,mean wt CN_20,mean wt CN_21,mean wt CN_22,mean wt CN_23,std_dev wt CN_23,mean wt CN_24
1917,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1827,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.089515,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,0.0,0.000000,0.0,0.0,0.000000,0.010073,0.000000,0.0,0.0,-0.016928,...,0.000007,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,0.0,0.000000,0.0,0.0,-0.088388,0.000000,0.029132,0.0,0.0,0.000000,...,0.006133,0.000245,0.000009,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1868,0.0,0.001776,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
out_soap_c = cross_validate(rf_soap_cv, df_soap_c, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [37]:
out_soap_c['test_score'].mean()

-0.7162662792077686

In [17]:
out_c_gbr = cross_validate(xgbr, df_soap_c, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [18]:
out_c_gbr['test_score'].mean()

-0.6287867176955302

# SOAP + Minor Featurisation

In [19]:
df_d = pd.read_pickle("df_d_mpe_gs.pkl")
df_d_reduced = df_d[df_d["defect_formation_energy"] > 0]

In [20]:
df_d_reduced = df_d_reduced.drop(["name","formula","mp_id", "host_structure", "defect_index", "defect_structure_relaxed", "defect_structure_original","defect_formation_energy"],axis=1)

# df_cm_reduced = corr_reduction(df_cm_reduced.corr(),0.9, df_cm_reduced)

In [21]:
df_soap_d = df_soap_reduced20.join(df_d_reduced)

In [22]:
df_soap_d = df_soap_d.drop(["crystal_system"], axis = 1)

In [39]:
df_soap_d.head()

,mean SOAP_2688,std_dev SOAP_2499,std_dev SOAP_4434,mean SOAP_4924,mean SOAP_4735,mean SOAP_1364,mean SOAP_320,std_dev SOAP_5898,std_dev SOAP_1764,mean SOAP_5470,...,std_dev SOAP_4428,std_dev SOAP_456,std_dev SOAP_449,density,vpa,packing fraction,max packing efficiency,spacegroup_num,crystal_system_int,is_centrosymmetric
1917,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,27.717178,5.744461,7.340130,11.959952,0.410482,0.300958,1,7,False
1827,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,-0.089515,...,0.0,40.513252,5.710964,5.662070,16.106118,0.733155,0.311252,1,7,False
352,0.0,0.000000,0.0,0.0,0.000000,0.010073,0.000000,0.0,0.0,-0.016928,...,0.0,46.292114,7.228235,5.551771,11.513054,0.685517,0.379453,1,7,False
91,0.0,0.000000,0.0,0.0,-0.088388,0.000000,0.029132,0.0,0.0,0.000000,...,0.0,49.100141,7.180411,5.085798,12.389093,0.794698,0.277284,8,6,False
1868,0.0,0.001776,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,39.588382,6.194458,6.587255,15.571968,0.641528,0.265092,1,7,False


In [24]:
out_soap_d = cross_validate(rf_soap_cv, df_soap_d, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True)


In [25]:
out_soap_d['test_score'].mean()

-0.6904659048948119

In [27]:
out_d_gbr = cross_validate(xgbr, df_soap_d, y, cv = kf, scoring = 'neg_root_mean_squared_error', return_estimator = True,error_score = 'raise')

/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/saaj/miniconda3/envs/matminer/lib/python3.8/site-packages/xgboost/data.py:250: Future

In [38]:
out_d_gbr['test_score'].mean()

-0.6323743470677583